<a href="https://colab.research.google.com/github/FabioMMaia/face_recognition_4models/blob/main/CNN_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tqdm import tqdm
import pickle
tqdm.pandas()

from skimage.color import rgb2gray
from sklearn.datasets import fetch_lfw_people
from skimage import data, exposure
semente = 42
import json

from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
from datetime import datetime
import pytz

In [ ]:
config_dir = '/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/config/'
model_save_dir = '/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/model/'
erro_dir = '/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/error/'
log_dir = '/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/log/'

num_classes = 50
tag = '_' + str(num_classes) + '_classes'

In [ ]:
# def write_log(s):
#     with open(pasta +'CNN_config.txt','a') as f:
#       print(s, file=f)

# model_name = 'AlexNet'
# ref = datetime.now().strftime("%Y%m%d")
# model_ref = model_name + '_' + ref

# resultadoFile = open(pasta + 'CNN_config.txt', 'w')
# data_atual = datetime.now(pytz.timezone('America/Bahia')).strftime("%d/%m/%Y %H:%M:%S")
# print('Execução em:', data_atual, file=resultadoFile)
# resultadoFile.close()

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
import pickle
from google.colab import drive
drive.mount('/content/drive')

#- Carrega Labels e Informações da Imagens

In [ ]:
def show_images_characteristics(dataset_images):
  aux_imgs = subselect.merge(dataset_images[['images']], left_index=True, right_index=True, how='inner')
  aux_imgs = aux_imgs.merge(celebs_attributes,  left_index=True, right_index=True, how='left')

  for coluna in aux_imgs.columns:
    if coluna not in ['id','class']:
      try:
        print(coluna, 'amostras positivas:', aux_imgs[aux_imgs[coluna]==True].shape[0])
        fig, axes = plt.subplots(1,5, figsize=(10,12))
        for (i,row), ax in zip(aux_imgs[aux_imgs[coluna]==True].head(5).iterrows(), axes.ravel()): 
          ax.axis('off')
          ax.imshow(row['images'])
        plt.show()
      except:
        pass

In [ ]:
def filter_by_n_class(n):
  topn_classes = labels['class'].value_counts().head(n).index
  keep_images = labels[(labels['class'].isin(topn_classes)) & ~(labels.index.get_level_values(0).isin(blurry_images))].index
  subselect = labels[labels['class'].isin(topn_classes)]

  print('foram retornadas imagens {} de pessoas {} o que corresponde a {}% da base'.format(
      subselect.shape[0], 
      len(subselect['class'].unique()),
      round(100*subselect.shape[0]/labels.shape[0],2)))
  
  return keep_images

In [ ]:
def filter_by_freq(ratio):
  assert ratio>0 and ratio<1, 'Ratio deve ser entre 0 e 1'
  import math
  keep_class = labels['class'].value_counts().cumsum()
  final_list = keep_class[:(keep_class <= math.ceil(labels.shape[0]*ratio)).sum() + 1].index
  subselect = labels[labels['class'].isin(final_list)]
  return subselect

In [ ]:
def get_labels():
  labels= pd.read_csv(r'/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/Data/identity_CelebA.txt', header=None)

  labels.rename(columns={0:'id'},inplace=True)

  labels['class'] = labels['id'].apply(lambda x:x.split()[1])
  labels['class'] = labels['class'].astype(int)
  labels['image'] = labels['id'].apply(lambda x:x.split()[0])

  labels.set_index('image', inplace=True)

  return labels

In [ ]:
def get_celeb_attributes():
  celebs_attributes = pd.read_csv(r'/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/Data/list_attr_celeba.csv')
  celebs_attributes.set_index('image_id', inplace=True)
  m_sample =  celebs_attributes.shape[0]
  print('Total de {} imagens'.format(m_sample))
  celebs_attributes=celebs_attributes.apply(lambda x:x>0)
  celebs_attributes =celebs_attributes.merge(labels, left_index=True, right_index=True, how ='inner')
  assert celebs_attributes.shape[0]==m_sample
  return celebs_attributes


In [ ]:
labels = get_labels()
celebs_attributes = get_celeb_attributes()

In [ ]:
blurry_images = celebs_attributes[(celebs_attributes['Blurry'])]

print('numero de amostras c/ caracteristicas:', blurry_images.shape[0] )
print('percentual da base:', blurry_images.shape[0]/labels.shape[0] )

blurry_images = blurry_images.index

In [ ]:
keep_images  = filter_by_n_class(num_classes)

In [ ]:
def load_images(save=False):
  import zipfile
  from tqdm import tqdm
  from PIL import Image
  from skimage.transform import resize

  imagens_path = '/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/CelebA/img_align_celeba.zip'
  imgzip = zipfile.ZipFile(imagens_path)
  inflist = imgzip.infolist()
  all_images = {}

  # Testando com imagens da classe 3

  for f in tqdm(inflist):
    file_name = f.filename.split('/')[1]
    if file_name in keep_images:
    # if '.jpg' in file_name:
      ifile = imgzip.open(f)
      img = Image.open(ifile)
      img_resize = resize(np.array(img), (128, 128))
      all_images[file_name] = img_resize

  return all_images

  if save:
    with open(r"/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/Pipeline/Data/images_raw_resized_top"+str(n)+".obj", "wb") as fp:
        pickle.dump(dataset_raw, fp)

        
all_images = load_images()   

In [ ]:
dataset_raw = pd.DataFrame({'img':all_images.keys(), 'images':all_images.values()})
dataset_raw.set_index('img', inplace=True)
dataset_raw = dataset_raw.merge(labels, left_index=True, right_index=True,how='left')
n_classes = len(dataset_raw['class'].unique())

In [ ]:
def show_images_class():
  for class_sample in dataset_raw['class'].unique()[:5]:
    fig, axes = plt.subplots(5,5, figsize=(10,12))
    print('class:{}'.format(class_sample))

    for (i,row), ax in zip(dataset_raw[dataset_raw['class']==class_sample][['images']].head(25).iterrows(), axes.ravel()): 
      ax.axis('off')
      ax.imshow(row['images'])  
    plt.show()

show_images_class()

In [ ]:
classes_n= dataset_raw['class'].unique()
num_classes = len(classes_n)
indexes = np.arange(0,len(classes_n))

conversor_classes = {classes_n[i]: indexes[i] for i in range(len(classes_n))}
conversor_classes_inv = {v: k for k, v in conversor_classes.items()}

dataset_raw['class_n'] = dataset_raw['class'].map(conversor_classes)

In [ ]:
images = np.array(list(dataset_raw['images'].values))
n_saidas = len(dataset_raw['class'].unique())
target = keras.utils.to_categorical(dataset_raw['class_n'].values, n_saidas).astype('int32')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, target, stratify=target, test_size=0.30, random_state=24)

In [ ]:
# Dimensões
img_lin, img_col, n_channels = X_train.shape[1], X_train.shape[2],X_train.shape[3]
input_shape = (img_lin, img_col, n_channels)
n_saidas = len(dataset_raw['class'].unique())

# Semente
# as sementes ajudam a ter resultados reproduzíveis
from numpy.random import seed
from tensorflow.random import set_seed
seed(semente)
set_seed(semente)

# vamos usar o mesmo número de épocas e batchsize para ambos
batch_size = 100
epochs = 10_000

# Criterio de parada
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)

# Loss
loss = 'categorical_crossentropy'
adam_optimizer = keras.optimizers.Adam()
metrics_evaluation = [tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.AUC()]

In [ ]:
def AlexNet_model():
  AlexNet = keras.models.Sequential()
  AlexNet.add(keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape= input_shape))
  AlexNet.add(keras.layers.BatchNormalization())
  AlexNet.add(keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
  AlexNet.add(keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
  AlexNet.add(keras.layers.BatchNormalization())
  AlexNet.add(keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)))
  AlexNet.add(keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
  AlexNet.add(keras.layers.BatchNormalization())
  AlexNet.add(keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
  AlexNet.add(keras.layers.BatchNormalization())
  AlexNet.add(keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
  AlexNet.add(keras.layers.BatchNormalization())
  AlexNet.add(keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2),padding='same'))
  AlexNet.add(keras.layers.Flatten())
  AlexNet.add(keras.layers.Dense(4096, activation='relu'))
  AlexNet.add(keras.layers.Dropout(0.5))
  AlexNet.add(keras.layers.Dense(4096, activation='relu'))
  AlexNet.add(keras.layers.Dropout(0.5))
  AlexNet.add(keras.layers.Dense(n_saidas, activation='softmax'))
  return AlexNet

AlexNet = AlexNet_model()


AlexNet.summary()

In [ ]:
train_datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=False,
        samplewise_std_normalization = False,
        shear_range=0.1,
        zoom_range=0.2,
        fill_mode='nearest',
        rotation_range = 45,
        )

test_datagen = ImageDataGenerator(
        samplewise_std_normalization = True)

train_gen = train_datagen.flow(X_train, y_train, batch_size=32)
test_gen = train_datagen.flow(X_test, y_test, batch_size=32)

def test_data_augmentation():
  fig, (ax1, ax2) = plt.subplots(2, 4, figsize = (12, 6))
  for c_ax1, c_ax2, (train_img, _), (test_img, _) in zip(ax1, ax2, train_gen, test_gen):
      c_ax1.imshow(train_img[0,:,:,:])
      c_ax1.set_title('Train Image')
      c_ax1.axis('off')

      c_ax2.imshow(test_img[0,:,:,:])
      c_ax2.set_title('Test Image')
      c_ax2.axis('off')

test_data_augmentation()

In [ ]:
AlexNet.compile(loss=loss,
              optimizer= adam_optimizer,
               metrics=metrics_evaluation)


hist_AlexNet = AlexNet.fit_generator(train_gen,
                      steps_per_epoch=10,
                      epochs = epochs, 
                      callbacks =[callback],
                      validation_data=test_gen, 
                      validation_steps=10)

# Salva os configs
with open(config_dir + 'CNN_AlexNet' + tag + '.json', 'w') as f:
    json.dump(AlexNet.get_config(), f)

# Salva os erros
with open(erro_dir + 'CNN_AlexNet' + tag + '.json', 'w') as f:
    json.dump(hist_AlexNet.history, f)

# Salva o modelo
AlexNet.save(model_save_dir + 'CNN_AlexNet' + tag )

In [ ]:
def plot_history(label, hist_obj, save=True):

  plt.plot(hist_obj.history['loss'])
  plt.plot(hist_obj.history['val_loss'])
  plt.title('Loss')
  plt.legend(["CNN - Treinamento", "CNN - Validação"], loc="best")
  if save:
    plt.savefig(erro_dir + label + '_' + tag + '_loss_graph'  + '.png')
  plt.show()

  plt.plot(hist_obj.history['categorical_accuracy'])
  plt.plot(hist_obj.history['val_categorical_accuracy'])
  plt.title('Acurácia')
  plt.legend(["CNN - Treinamento", "CNN - Validação"], loc="best")
  if save:
    plt.savefig(erro_dir +  label + '_' + tag + '_accuracy'  + '.png')
  plt.show()

plot_history('CNN_AlexNet', hist_AlexNet)

In [ ]:
print("Treinamento:")
score_tr = AlexNet.evaluate(X_train, y_train, verbose = 1, batch_size=batch_size)
print("Teste:")
score_Te = AlexNet.evaluate(X_test, y_test, verbose = 1, batch_size=batch_size)

# Salva performance pro log

log={}
log['CNN_AlexNet' + tag] = {}
log['CNN_AlexNet' + tag]['metrics'] = {'loss_train':score_tr[0], 'categorical_accuracy_train':score_tr[1], 'auc_1_train':score_tr[2],
       'loss_val':score_Te[0], 'categorical_accuracy_val':score_Te[1], 'auc_1_val':score_Te[2]}

# Salva as metricas
with open(log_dir + 'CNNs' + tag + '.json', 'w') as f:
    json.dump(log, f)

# Transfer Learning e MobileNet

In [ ]:
import tensorflow as tf

base_model =  tf.keras.applications.MobileNetV2(input_shape=X_train.shape[1:],
                                               include_top=True,
                                               weights='imagenet')

base_model.summary()

nb_layers = len(base_model.layers)
print(base_model.layers[nb_layers - 2].name)
print(base_model.layers[nb_layers - 1].name)

In [ ]:
def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip('horizontal'))
    data_augmentation.add(RandomRotation(0.2))
    
    return data_augmentation


def celeb_mobel(data_augmentation=data_augmenter(), preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input):
    ''' Cria modelo a partir modelo baseline
    Arguments:
        data_augmentation -- função de data augmentation
        preprocess_input -- função de preprocessamento

    Retorna:
        tf.keras.model
    '''
        
    base_model = tf.keras.applications.MobileNetV2(input_shape=X_train.shape[1:],
                                                   include_top=False, 
                                                   weights='imagenet')
    
    # Congela os pesos do modelo
    # base_model.trainable = False 

    # Cria input conforme as dimensões de entrada (deve ser o mesmo da FaceNet)
    inputs = tf.keras.Input(shape=input_shape) 
    
    # Aplica data augmentation nos inputs
    x = data_augmentation(inputs)
    
    # Preprocessamento usando os mesmo pesos que o modelo foi treinado
    x = preprocess_input(x) 
    
    # Coloca training to False para não mostrar estatística do treino
    x = base_model(x, training=True) 
    
    # Usa average pooling para sumarizar as informações dos canais
    x = tf.keras.layers.GlobalAveragePooling2D()(x) 

    # include dropout c/  p=0.2 (evitar sobreajuste)
    x = tf.keras.layers.Dropout(rate=0.2)(x)
        
    # Adiciona a camada de saída
    outputs =  tf.keras.layers.Dense(num_classes, activation='softmax')(x)
        
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [ ]:
CelebModel = celeb_mobel()

In [ ]:
CelebModel.summary()

In [ ]:
CelebModel.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(),
               metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.AUC()])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200) # Parada

hist_CelebModel = CelebModel.fit(X_train, y_train,
                    batch_size=batch_size,
                    validation_split=0.1,
                    use_multiprocessing= True,
                    callbacks=[callback],
                    epochs=epochs, 
                    validation_data=(X_test, y_test),
                    verbose=1)


# Salva os configs
with open(config_dir + 'CNN_Celeb' + tag + '.config', 'wb') as f:
    pickle.dump(CelebModel.get_config(), f)

# Salva os erros
with open(erro_dir + 'CNN_Celeb' + tag + '.json', 'w') as f:
    json.dump(hist_CelebModel.history, f)

# Salve o modelo
CelebModel.save(model_save_dir + 'CNN_Celeb' + tag )

In [ ]:
# Plotta graficos
plot_history('CelebModel',hist_CelebModel)

In [ ]:
# Valida
print("Treinamento:")
score_tr = CelebModel.evaluate(X_train, y_train, verbose = 1, batch_size=batch_size)
print("Teste:")
score_Te = CelebModel.evaluate(X_test, y_test, verbose = 1, batch_size=batch_size)

log['CelebModel' + tag] = {}
log['CelebModel' + tag]['metrics'] = {'loss_train':score_tr[0], 'categorical_accuracy_train':score_tr[1], 'auc_1_train':score_tr[2],
       'loss_val':score_Te[0], 'categorical_accuracy_val':score_Te[1], 'auc_1_val':score_Te[2]}

# Salva as metricas
with open(log_dir + 'CNNs' + tag + '.json', 'w') as f:
    json.dump(log, f)

In [ ]:
def print_resultados(df, analise_pred, acertos=True):

  for i,row in analise_pred.query('acerto==@acertos').head(20).iterrows():
    fig, ax = plt.subplots(1,3, figsize=(10,6))

    ax[0].axis('off')
    ax[0].imshow(row['images'], cmap=plt.cm.gray)
    ax[0].set_title('image')

    real = df[df['class']==row['class']]['images'][0]
    predicted = df[df['class']==row['class_pred']]['images'][0]

    ax[1].axis('off')
    ax[1].imshow(real, cmap=plt.cm.gray)
    ax[1].set_title('real: person:{}'.format(row['class']))

    ax[2].axis('off')
    ax[2].imshow(predicted, cmap=plt.cm.gray)
    ax[2].set_title('pred (probs:{}) :person:{}'.format(row['class_pred_proba'], row['class_pred']))

    plt.show()

## Analise de Erros e Acertos

In [ ]:
analise_pred = pd.Series(y_test.argmax(axis=1)).map(conversor_classes_inv).to_frame().rename(columns={0:'class'}).merge(dataset_raw[~dataset_raw['class_n'].duplicated(keep='first')], 
                                                                                                         left_on='class',
                                                                                                         right_on='class',
                                                                                                         how='left')
predictions_test = CelebModel.predict(X_test)
analise_pred['class_pred'] = predictions_test.argmax(axis=1)
analise_pred['class_pred']= analise_pred['class_pred'].map(conversor_classes_inv)
analise_pred['class_pred_proba'] =np.max(predictions_test, axis=1)
analise_pred['class_pred_proba']=analise_pred['class_pred_proba'].apply(lambda x:round(x,2))
analise_pred['acerto'] = analise_pred['class']== analise_pred['class_pred']

print('acertos')
print_resultados(dataset_raw, analise_pred, acertos=True)

print('erros')
print_resultados(dataset_raw, analise_pred, acertos=False)

In [ ]:
def test_other_samples(n_images=10):
  imagens_path = '/content/drive/MyDrive/Projeto ML/2022/Aprendizado de Maquinas/Projeto/CelebA/img_align_celeba.zip'
  imgzip = zipfile.ZipFile(imagens_path)
  inflist = imgzip.infolist()
  all_images = {}

  # Testando com imagens da classe 3
  it=0
  for f in tqdm(inflist):
    file_name = f.filename.split('/')[1]
    if file_name in keep_images:
      it+=1
    # if '.jpg' in file_name:
      ifile = imgzip.open(f)
      img = Image.open(ifile)
      img_resize = resize(np.array(img), (128, 128))
      all_images[file_name] = img_resize

      if it>n_images:
        return all_images



## Analise de amostras aleatórias do universo

In [ ]:
import zipfile
from PIL import Image
from skimage.transform import resize

samples = test_other_samples(40)
samples_test = np.array(list(samples.values()))
preds= CelebModel.predict(samples_test)
out = pd.Series(preds.argmax(axis=1)).map(conversor_classes_inv)

In [ ]:
df_analise = dataset_raw[dataset_raw['class'].isin(out.values)]
for cont, (i,r) in enumerate(df_analise[~df_analise['class'].duplicated(keep='first')].iterrows()):
  fig, axis = plt.subplots(1,2,figsize=(5,4))
  axis[0].imshow(samples_test[cont])
  axis[0].set_title('image')
  axis[0].axis('off')
  axis[1].imshow(r['images'])
  axis[1].set_title('pred')
  axis[1].axis('off')
  plt.show()